In [1]:
import os
import pandas as pd

def process_all_files(root_dir, matches_info_df):
    all_dfs = []

    for subdir, dirs, files in os.walk(root_dir):
        bz2_files = [f for f in files if f.endswith('.bz2')]

        if bz2_files:
            temp_dfs = []

            for file in bz2_files:
                file_path = os.path.join(subdir, file)
                temp_df = parse_file_to_dataframe(file_path, matches_info_df)

                if temp_df is not None:
                    temp_dfs.append(temp_df)

            if temp_dfs:
                dir_df = pd.concat(temp_dfs)
                all_dfs.append(dir_df)

    if all_dfs:  # Check if all_dfs is not empty
        # Concatenate all dataframes
        final_df = pd.concat(all_dfs).reset_index(drop=True)
        # Group by match_id and find the market_id with the maximum count for each group
        max_market_id_per_match = final_df.groupby('match_id')['marketId'].apply(
            lambda x: x.mode().iloc[0]
        ).reset_index().rename(columns={'marketId': 'max_marketId'})
        # Merge to associate max marketId with the final_df
        final_df = final_df.merge(max_market_id_per_match, on='match_id')
        # Filter final_df for rows where marketId matches the max_marketId
        final_df = final_df[final_df['marketId'] == final_df['max_marketId']]
        final_df = final_df.drop(columns=['max_marketId'])  # Drop the auxiliary column
    else:
        # Return an empty DataFrame if no match found
        final_df = pd.DataFrame()

    return final_df

In [2]:
import bz2
import json
import pandas as pd
from difflib import SequenceMatcher
from collections import Counter

def is_partial_match(str1, str2, threshold=0.75):
    """Check if two strings are a partial match above a certain threshold."""
    return SequenceMatcher(None, str1, str2).ratio() > threshold

def find_matching_match_name(raw_match_name, matches_info_df):
    """Find a matching match name in matches_info_df using partial match."""
    for match_name in matches_info_df['match_name']:
        if is_partial_match(raw_match_name, match_name):
            return match_name
    return None

def parse_file_to_dataframe(bz2_file_path, matches_info_df):
    with bz2.open(bz2_file_path, 'rt') as file:
        data = [json.loads(line) for line in file]
        
    # Extract and find a matching match name
    raw_match_name = data[0].get("mc", [{}])[0].get("marketDefinition", {}).get("eventName")
    if not raw_match_name:
        return None

    valid_match_name = find_matching_match_name(raw_match_name, matches_info_df)
    if not valid_match_name:
        return None

    # Collect all publish dates from the data
    publish_dates = [pd.to_datetime(item.get("pt"), unit='ms').date() for item in data]

    # Find the date that occurs the most frequently
    most_common_date, _ = Counter(publish_dates).most_common(1)[0]

    # Match the most common date with the date from matches_info_df corresponding to the matched name
    match_info = matches_info_df[matches_info_df['match_name'] == valid_match_name]
    matched_date = match_info[match_info['match_date'] == most_common_date]
    # If there is no match, return None
    if matched_date.empty:
        return None

    # Extract match_id for the matched date
    match_id = matched_date['match_id'].iloc[0]
    
    # Extract the time of the match to filter betting after the match started
    match_event_dt = pd.to_datetime(matched_date.publishTime)

    # Parsing data
    parsed_data = []
    team_names = {}

    for item in data:
        market_id = item.get("mc", [{}])[0].get("id")
        publish_time = item.get("pt")

        # Update team names if available
        if 'marketDefinition' in item.get("mc", [{}])[0]:
            runner_status_data = item.get("mc", [{}])[0].get("marketDefinition", {}).get("runners", [])
            for runner in runner_status_data:
                runner_id = runner.get("id")
                if runner_id is not None:
                    team_names[runner_id] = runner.get("name")

        runners_data = item.get("mc", [{}])[0].get("rc", [])
        for runner in runners_data:
            runner_id = runner.get("id")
            last_price_traded = runner.get("ltp")
            if runner_id is not None and market_id and publish_time:
                parsed_data.append({
                "marketId": market_id,
                "publishTime": publish_time,
                "selection_selectionId": runner_id,
                "selection_lastPriceTraded": last_price_traded,
                "match_name": valid_match_name, # Use the matched match name
                "team_name": team_names.get(runner_id, "Unknown"), # Default to "Unknown" if not found
                "match_id": match_id # Use the matched match_id
                })
    # Create and filter DataFrame to include only rows with the most common date
    try:
        df = pd.DataFrame(parsed_data)
        df['publishTime'] = pd.to_datetime(df['publishTime'], unit='ms')
        df = df[df['publishTime'].dt.date == most_common_date]
        
        # Filtering betting records after the match started
        df = df[df['publishTime'] > match_event_dt.iloc[0]]
        
    except:
        return None

    return df.reset_index(drop=True)

In [3]:
# Filtering matches available in the events dataset
matches_date_df = pd.read_csv('matches_data.csv')
matches_date_df['publishTime'] = pd.to_datetime(matches_date_df['match_date'])
matches_date_df['match_date'] = matches_date_df['publishTime'].dt.date

root_dir = "D:/ARU Modules/Final Project/dataset/betting_dataset/"
final_dataframe = process_all_files(root_dir, matches_date_df)

### Mapping Home/Away Teams

In [12]:
"""For this we need the 'map_home_away_team.csv' file form the events processing notebook.
This file contains the information about the home/away team along with the match_id and match_date"""

home_away_df = pd.read_csv("processed_df/map_home_away_team.csv")

In [13]:
# Merge the DataFrames on 'match_id'
merged_df = pd.merge(final_dataframe, home_away_df[['home_team', 'away_team', 'match_id']], on='match_id', how='left')

In [15]:
'''When applyig the price mapping to home and away teams, there are some differences in names due to this reason
the mapping is giving NA to some rows for which the team_name and home/away_team is not matching. To solve this we will manually change the names in the
team_name to align wiht the home/away_team names'''

# Define a dictionary with original names as keys and full names as values
name_mappings = {
    'Man City': 'Manchester City',
    'Bournemouth': 'AFC Bournemouth',
    'Brighton': 'Brighton & Hove Albion',
    'Man Utd': 'Manchester United',
    'C Palace': 'Crystal Palace',
    'Huddersfield': 'Huddersfield Town',
    'Tottenham': 'Tottenham Hotspur',
    'West Brom': 'West Bromwich Albion',
    'Newcastle': 'Newcastle United',
    'Swansea': 'Swansea City',
    'Huddersfield Town Town': 'Huddersfield Town',
    'Leicester': 'Leicester City',
    'Leicester City City': 'Leicester City',
    'West Ham': 'West Ham United',
    'Stoke': 'Stoke City',
    'Stoke City City': 'Stoke City'
}

# Loop through the dictionary and replace each key with its corresponding value in 'team_name'
for original_name, full_name in name_mappings.items():
    merged_df['team_name'] = merged_df['team_name'].str.replace(original_name, full_name)


In [16]:
def process_betting_data(df):
    df['publishTime'] = pd.to_datetime(df['publishTime'], format='%M:%S.%f')
    
    df['home_price'] = pd.NA
    df['away_price'] = pd.NA
    df['draw_price'] = pd.NA

    def process_group(group):
        group = group.sort_values(by='publishTime')
        last_prices = {'home': pd.NA, 'away': pd.NA, 'draw': pd.NA}
        
        for index, row in group.iterrows():
            team_name = row['team_name']
            home_team = row['home_team']
            away_team = row['away_team']
            
            # Update the last known price based on the team name
            if row['team_name'] == 'The Draw':
                last_prices['draw'] = row['selection_lastPriceTraded']
            elif row['team_name'] == row['home_team']:
                last_prices['home'] = row['selection_lastPriceTraded']
            elif row['team_name'] == row['away_team']:
                last_prices['away'] = row['selection_lastPriceTraded']
            
            group.at[index, 'home_price'] = last_prices['home']
            group.at[index, 'away_price'] = last_prices['away']
            group.at[index, 'draw_price'] = last_prices['draw']

        group[['home_price', 'away_price', 'draw_price']] = group[['home_price', 'away_price', 'draw_price']].fillna(method='ffill').fillna(method='bfill')
        
        return group

    processed_data = df.groupby('match_id').apply(process_group).reset_index(drop=True)
    
    return processed_data


# Applying the function
processed_dataset = process_betting_data(merged_df)
processed_dataset.drop_duplicates(subset="publishTime", keep="first", inplace=True) # To remove the repeating prices

C:\Users\Ashutosh\AppData\Local\Temp\ipykernel_18040\1616611258.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group[['home_price', 'away_price', 'draw_price']] = group[['home_price', 'away_price', 'draw_price']].fillna(method='ffill').fillna(method='bfill')
C:\Users\Ashutosh\AppData\Local\Temp\ipykernel_18040\1616611258.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group[['home_price', 'away_price', 'draw_price']] = group[['home_price', 'away_price', 'draw_price']].fillna(method='ffill').fillna(method='bfill')
C:\Users\Ashutosh\AppData\Local\Temp\ipykernel_18040\1616611258.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future 

In [42]:
processed_dataset.to_csv("processed_df/betting_df.csv", index = False)